In [1]:
import cassandra
from cassandra.cluster import Cluster, ConnectionException

In [2]:
nodes = ['127.0.0.1'] # list of cluster devices

In [3]:
cluster = Cluster(nodes)

In [4]:
def getClusterSession(cluster):
    try:        
        session = cluster.connect()
        
        print('Connection established successfully.')
        return session
        
    except(ConnectionException, Exception) as exception :
        print ("Error while connecting to Cassandra Cluster\n", exception)
            

def closeSession(session):
    #closing database connection.
    if(session):
        session.shutdown()
        print("Cassandra connection is closed")

In [5]:
session = getClusterSession(cluster)
closeSession(session)

Connection established successfully.
Cassandra connection is closed


In [9]:
def createKeyspace(cluster):
    command = '''
                CREATE KEYSPACE IF NOT EXISTS Employees
                WITH REPLICATION = 
                {'class': 'SimpleStrategy', 'replication_factor' : 1}
                '''
    session = getClusterSession(cluster)
    
    try:
        
        session.execute(command)
        print('Keyspace Successfully Created.')
        
    except(Exception) as error :
        print ("Error while creating keyspace : ", error)
    finally:
        closeSession(session)

In [10]:
createKeyspace(cluster)

Connection established successfully.
Keyspace Successfully Created.
Cassandra connection is closed


In [11]:
def createEmployeeTable(cluster, keyspace):
    command = '''
                CREATE TABLE IF NOT EXISTS Employee(
                    EmployeeID INT,
                    EmployeeName TEXT,
                    ManagerID INT,
                    YearJoined INT,
                    PRIMARY KEY(YearJoined, EmployeeID)
                );'''
    session = getClusterSession(cluster)
    session.set_keyspace(keyspace)
    try:
        
        session.execute(command)
        print('Table Successfully Created.')
        
    except(Exception) as error :
        print ("Error while creating table.", error)
    finally:
        closeSession(session)
    

In [12]:
createEmployeeTable(cluster, 'employees')

Connection established successfully.
Table Successfully Created.
Cassandra connection is closed


In [13]:
def insertEmployee(cluster, keyspace, record):
    command = 'INSERT INTO Employee (EmployeeID, EmployeeName, YearJoined, ManagerID) VALUES (%s, %s, %s, %s)'
    
    session = getClusterSession(cluster)
    session.set_keyspace(keyspace)
    
    try:
        
        session.execute(command, record)
        print('Records inserted successfully.')
        
    except(Exception) as error :
        print ("Error while inserting data.", error)
    finally:
        closeSession(session)

In [16]:
# insertEmployee(cluster, 'employees', (1, 'Ali', 2013, None))
# insertEmployee(cluster, 'employees', (2, 'Khan', 2009, None))

Connection established successfully.
Records inserted successfully.
Cassandra connection is closed


In [23]:
def retrieveEmployees(cluster, keyspace, employeeId=''):
    
    session = getClusterSession(cluster)
    session.set_keyspace(keyspace)
    
    if employeeId:
        command = 'SELECT * FROM Employee WHERE EmployeeID = %s;'
        try:
            
            employees_ = session.execute(command, employeeId)
        
            return employees_
        
        except(Exception) as error:
            print ("Unable to retrieve data. ", error)
        finally:
            closeSession(session)
            
    else:
        
        command = 'SELECT * FROM Employee;'
        try:
            employees_ = session.execute(command)
        
            return employees_
        
        except(Exception) as error:
            print ("Unable to retrieve data. ", error)
        finally:
            closeSession(session)

In [35]:
rows = retrieveEmployees(cluster, 'employees')

Connection established successfully.
Cassandra connection is closed


In [36]:
for row in rows:
    print(row.employeeid, row.employeename, row.yearjoined)

2 Khan 2009
1 Ali 2013


In [37]:
cluster.shutdown()